<a href="https://colab.research.google.com/github/DrPaulaEkene/Uterine-Myoma-Uterine-fibroids-Ultrasound_image-classification/blob/main/Uterine_Myoma_(Uterine_fibroids)_Ultrasound_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook focuses on the Comparative Analysis of Deep Learning Architectures for Uterine Myoma Classification: ResNet101 vs. ResNet101 plus DCGAN**

*Import all needed libraries*

In [1]:
# Make all necessary imports
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
# Import the drive module
from google.colab import drive

# Mount the Google Drive to the Colab environment.
drive.mount('/content/drive')

Mounted at /content/drive


Merge directories and prepare dataset

In [3]:
# Create the data file paths/directories
data_directory = "/content/drive/MyDrive/UF_dataset"
train_directory = "/content/drive/MyDrive/UF_dataset/train"
test_directory = "/content/drive/MyDrive/UF_dataset/test"
print(os.listdir(data_directory))

['1.2.826.0.1.3680043.2.461.11301520.1540965563.jpg', '1.2.826.0.1.3680043.2.461.11301518.1849713856.jpg', '1.2.826.0.1.3680043.2.461.11301519.531737451.jpg', '1.2.826.0.1.3680043.2.461.11301521.1331253811.jpg', '1.2.826.0.1.3680043.2.461.11301521.250905338.jpg', '1.2.826.0.1.3680043.2.461.11301521.704960520.jpg', '1.2.826.0.1.3680043.2.461.11301521.1865503937.jpg', '1.2.826.0.1.3680043.2.461.11301521.4216844312.jpg', '1.2.826.0.1.3680043.2.461.11301522.742552078.jpg', '1.2.826.0.1.3680043.2.461.11301522.2857734559.jpg', '1.2.826.0.1.3680043.2.461.11301522.1372347287.jpg', '1.2.826.0.1.3680043.2.461.11301521.2436177613.jpg', '1.2.826.0.1.3680043.2.461.11301520.2658168329.jpg', '1.2.826.0.1.3680043.2.461.11301522.913479745.jpg', '1.2.826.0.1.3680043.2.461.11301525.2220413095.jpg', '1.2.826.0.1.3680043.2.461.11301523.1653786924.jpg', '1.2.826.0.1.3680043.2.461.11301523.2948715458.jpg', '1.2.826.0.1.3680043.2.461.11301524.2960341954.jpg', '1.2.826.0.1.3680043.2.461.11301525.2072747749.jpg

In [4]:
# Merge directories to enable a flexible split
# Define the source directories for train and test datasets
train_dir = train_directory
test_dir = test_directory

# Define the target directories within drive
combined_uf_dir = '/content/drive/MyDrive/UMdataset/UF'
combined_nuf_dir = '/content/drive/MyDrive/UMdataset/NUF'

# Create the target directories
os.makedirs(combined_uf_dir, exist_ok=True)
os.makedirs(combined_nuf_dir, exist_ok=True)

def copy_files(src_dir, dst_dir):
    for root, _, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.jpg'):
                shutil.copy(os.path.join(root, file), os.path.join(dst_dir, file))

# Copy UF files
copy_files(os.path.join(train_dir, 'UF'), combined_uf_dir)
copy_files(os.path.join(test_dir, 'UF'), combined_uf_dir)

# Copy NUF files
copy_files(os.path.join(train_dir, 'NUF'), combined_nuf_dir)
copy_files(os.path.join(test_dir, 'NUF'), combined_nuf_dir)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the dataset directories
dataset_dir = '/content/drive/MyDrive/UMdataset'
uf_dir = os.path.join(dataset_dir, 'UF')
nuf_dir = os.path.join(dataset_dir, 'NUF')

# Function to split data and copy files to destination directory
def split_and_copy_data(class_dir, output_dir, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    # Create output directories if they don't exist
    os.makedirs(output_dir, exist_ok=True)
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'validation')
    test_dir = os.path.join(output_dir, 'test')
    for dir in [train_dir, val_dir, test_dir]:
        os.makedirs(dir, exist_ok=True)
        os.makedirs(os.path.join(dir, 'UF'), exist_ok=True)  # Create UF subfolder
        os.makedirs(os.path.join(dir, 'NUF'), exist_ok=True)  # Create NUF subfolder

    # Function to copy files to destination directory
    def copy_files(src_files, dst_dir):
        for src_file in src_files:
            dst_subfolder = os.path.join(dst_dir, 'UF' if 'UF' in src_file else 'NUF')
            os.makedirs(dst_subfolder, exist_ok=True)
            shutil.copy(os.path.join(class_dir, src_file), os.path.join(dst_subfolder, src_file))

    # Split data into train, validation, test sets
    files = [file for file in os.listdir(class_dir) if file.endswith('.jpg')]
    train_files, test_files = train_test_split(files, test_size=test_ratio, random_state=42)
    train_files, val_files = train_test_split(train_files, test_size=val_ratio/(train_ratio + val_ratio), random_state=42)

    # Copy train files
    copy_files(train_files, train_dir)

    # Copy validation files
    copy_files(val_files, val_dir)

    # Copy test files
    copy_files(test_files, test_dir)

    print("Split datasets saved successfully.")

# Split and copy UF data
split_and_copy_data(uf_dir, dataset_dir)

# Split and copy NUF data
split_and_copy_data(nuf_dir, dataset_dir)


Split datasets saved successfully.


In [37]:
# Define functions to count UF and NUF files
def count_files(files):
    uf_count = sum(1 for file in files if 'UF' in file)
    nuf_count = sum(1 for file in files if 'NUF' in file)
    return uf_count, nuf_count

# Count UF and NUF files in train set
train_uf_count, train_nuf_count = count_files(train_files)
print("Train set - UF: {}, NUF: {}".format(train_uf_count, train_nuf_count))

# Count UF and NUF files in validation set
val_uf_count, val_nuf_count = count_files(val_files)
print("Validation set - UF: {}, NUF: {}".format(val_uf_count, val_nuf_count))

# Count UF and NUF files in test set
test_uf_count, test_nuf_count = count_files(test_files)
print("Test set - UF: {}, NUF: {}".format(test_uf_count, test_nuf_count))


Train set - UF: 1399, NUF: 787
Validation set - UF: 201, NUF: 113
Test set - UF: 400, NUF: 225


Create datagenerators to efficiently load, preprocess, and augment images during training

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define directories
train_dir = '/content/drive/MyDrive/UMdataset/train'
validation_dir = '/content/drive/MyDrive/UMdataset/validation'
test_dir = '/content/drive/MyDrive/UMdataset/test'

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 1399 images belonging to 2 classes.
Found 201 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/10
28/43 [==================>...........] - ETA: 10:08 - loss: 0.0389 - accuracy: 0.9684

In [ ]:
# Define ResNet101 model
base_model = ResNet101(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print(f"Test accuracy: {test_accuracy:.4f}")